$ \mathrm{\overline{C}_{D}}_{t_0}^{t_N}(\mathbf{x}_0) $ is given by:

\begin{equation}
\mathrm{\overline{C}_{D}}_{t_0}^{t_N}(\mathbf{x}_0) = \dfrac{1}{t_N-t_0} \int_{t_0}^{t_N}\text{det }[\mathbf{D}(\mathbf{F}_{t_0}^t(\mathbf{x}_0), t)] [\mathbf{T}_{t_0}^t(\mathbf{x}_0)]^{-1}dt 
\end{equation}

and denotes the time-averaged, diffusivity structure-weighted version of the classic right Cauchy–Green strain tensor. 

The transport tensor $ \mathrm{T_{t_0}^{t}}(\mathbf{x}_0) $ is given by:

\begin{equation}
\mathrm{T_{t_0}^{t}}(\mathbf{x}_0) = [\nabla \mathbf{F}_{t_0}^t]^{-1}(\mathbf{x}_0) \mathbf{D}(\mathbf{F}_{t_0}^t(\mathbf{x}_0), t)) [\nabla \mathbf{F}_{t_0}^t]^{-T}(\mathbf{x}_0).
\end{equation}


It thus holds:

\begin{equation}
\mathrm{\overline{C}_{D}}_{t_0}^{t_N}(\mathbf{x}_0) = \dfrac{1}{t_N-t_0} \int_{t_0}^{t_N}\text{det }[\mathbf{D}(\mathbf{F}_{t_0}^t(\mathbf{x}_0), t)] [\nabla \mathbf{F}_{t_0}^t]^{T}(\mathbf{x}_0)\mathbf{D}(\mathbf{F}_{t_0}^t(\mathbf{x}_0), t)^{-1}[\nabla \mathbf{F}_{t_0}^t(\mathbf{x}_0)] dt
\end{equation}

$ \mathbf{D}(\mathbf{x}(t), t) = \mathbf{D}(\mathbf{x}(t), t)^T = \begin{pmatrix} D_{11} && D_{12} \\ D_{12} && D_{22} \end{pmatrix} $ denotes a continuously differentiable, positive definite diffusion-structure tensor, with possible anisotropy, spatial and temporal variation. $ \mathbf{D}(\mathbf{x}(t), t) $ must also be an objective tensor. In the following notebooks we will omit spatial and temporal variations of the $ D(\mathbf{x}(t), t) $ and assume it to be constant. For isotropic diffusion ($ \mathbf{D}(\mathbf{x}(t), t) = \mathbf{I} $) it holds:

\begin{equation}
\mathrm{T_{t_0}^{t}}(\mathbf{x}_0) = [\mathbf{C}_{t_0}^t]^{-1}(\mathbf{x}_0).
\end{equation}

| Name | Type (Shape) | Description |
| --- | --- | --- |
| Interpolant_D11 | Interpolant object | Interpolant for $ D_{11} $|
| Interpolant_D22 | Interpolant object | Interpolant for $ D_{22} $|
| Interpolant_D12 | Interpolant object | Interpolant for $ D_{12} $|
| time | array (N, ) | time |
| Fmap | array (N,2) | flow map |
| gradFmap | array (2, 2, N) | gradient of flow-map |
| lenT | float | $ t_N-t_0 $ |
| n_averaging | int | approximate the average with N/n_averaging  points |
| T_avg | array (2, 2) | transport tensor $ \mathrm{\overline{T}_{t_0}^{t_N}}(\mathbf{x}_0) $|

In [4]:
# Import numpy
import numpy as np

In [6]:
def DiffusionCauchyGreen(Interpolant_D11, Interpolant_D12, Interpolant_D22, time, Fmap, gradFmap, lenT, n_averaging = 10):
    
    #evaluate the integral at these indices
    indices = range(0, gradFmap.shape[2], n_averaging)
    
    # Diffusive Cauchy Green
    Cd = np.zeros((2,2,len(indices))) * np.nan 
    
    
    counter = 0
    
    for i in indices:
        
        # evaluate D11 at x(t)
        D11 = Interpolant_D11([Fmap[1, i], Fmap[0, i], time[i]])[0]
        
        # evaluate D12 at x(t)
        D12 = Interpolant_D12([Fmap[1, i], Fmap[0, i], time[i]])[0]
        
        # evaluate D22 at x(t)
        D22 = Interpolant_D22([Fmap[1, i], Fmap[0, i], time[i]])[0]

        # determinant of D
        detD = D11*D22 - D12**2
        
        D = np.array([[D11, D12], [D12, D22]])

        # diffusion averaged CG tensor
        Cd[:,:,counter] = detD*(gradFmap[:,:,i].transpose()@(D@gradFmap[:,:,i]))
        
        counter+=1
    return np.average(Cd, axis = 2)